1. data로드를 통한 데이터 프레임 완성
2. 한글에 맞게 형태소분석기를 이용한 단어 토큰화 작업후 TF-IDF를 계산
3. 코사인 유사도 기반을 통한 영화 3종 추천

데이터 로드

In [1]:
import pandas as pd
data = pd.read_csv('20220511_data2.csv',low_memory=False)
data

,name,genre,content
0,올드보이,스릴러,술 좋아하고 떠들기 좋아하는 오.대.수. 본인의 이름풀이를 오늘만 대충 수습하며 살...
1,노트북,로맨스,"17살, ‘노아’는 밝고 순수한 ‘앨리’를 보고 첫눈에 반한다. 빠른 속도로 서로에..."
2,미션임파서블,액션,"도난당한 핵무기를 되찾아 오는 위험천만한 작전 중, 에단 헌트(톰 크루즈)는 임무를..."
3,니모를찾아서,애니메이션,“디즈니`픽사 최고의 애니메이션이 3D로 다시 찾아온다!” 호기심 가득한 아기 물고...
4,탑건,액션,해군 최신 전투기 F-14기를 모는 젊은 조종사 매버릭 대위(톰 크루즈)는 최고의 ...
5,친절한금자씨,스릴러,주변 사람들의 시선을 단번에 사로잡을 만큼 뛰어난 미모의 소유자인 금자(이영애)는 ...
6,반지의제왕,판타지,모든 힘을 지배할 악의 군주 ‘사우론’의 절대반지가 깨어나고 악의 세력이 세상을 지...
7,신세계,범죄,경찰청 수사 기획과 강과장(최민식)은 국내 최대 범죄 조직인 '골드문'이 기업형 조...
8,아저씨,NaN,불행한 사건으로 아내를 잃고 세상을 등진 채 전당포를 꾸려가며 외롭게 살아가는 전직...
9,해리포터,NaN,해리 포터(다니엘 래드클리프 분)는 위압적인 버논 숙부(리챠드 그리피스 분)와 냉담...


형태소분석기 이용

In [2]:
from konlpy.tag import Okt
tw = Okt()
data_l = [''.join(tw.morphs(i)) for i in data['content']]
type(data_l[0])

str

TF-IDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_t = tfidf.fit_transform(data_l)
t_idx = dict(zip(data['name'],data.index))
t_idx

{'올드보이': 0,
 '노트북': 1,
 '미션임파서블': 2,
 '니모를찾아서': 3,
 '탑건': 4,
 '친절한금자씨': 5,
 '반지의제왕': 6,
 '신세계': 7,
 '아저씨': 8,
 '해리포터': 9,
 '장고': 10,
 '어바웃 타임': 11,
 '스타트렉': 12,
 '트랜스포머': 13,
 '지금만나러갑니다': 14,
 '007 노타임투다이': 15,
 '인터스텔라': 16,
 '스타워즈': 17}

코사인 유사도, 맨하튼 거리, 유클리드 거리

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics import euclidean_distances
cos = cosine_similarity(tfidf_t,tfidf_t) # 숫자(점수)가 높으면 유사도도 높다.
c1=manhattan_distances(tfidf_t,tfidf_t)# 숫자(거리)가 높으면 유사도는 낮다
c2=euclidean_distances(tfidf_t,tfidf_t)

In [5]:
t_idx=dict(zip(data['name'],data.index))
def ck_s_t(t,cosine_sim=cos,k=0):
    idx=t_idx[t]
    c_sc=list(enumerate(cosine_sim[idx]))
    if k==0:
        c_sc=sorted(c_sc,key=lambda x:x[1],reverse=True)#내림차순 정렬
    else:
        c_sc=sorted(c_sc,key=lambda x:x[1])
    m_i=c_sc[1:4]
    m_idx=[i[0] for i in m_i]
    return data['name'].iloc[m_idx]

In [6]:
ck_s_t('올드보이',k=0) 

6     반지의제왕
1       노트북
2    미션임파서블
Name: name, dtype: object

In [7]:
ck_s_t('올드보이',c1,k=1)

13    트랜스포머
8       아저씨
16    인터스텔라
Name: name, dtype: object

In [8]:
ck_s_t('올드보이',c2,k=1) 

6     반지의제왕
7       신세계
3    니모를찾아서
Name: name, dtype: object